# CIFAR 10 Classifier: Pytorch Lightning Version

This notebook uses Pytorch Lightning (PL) to demonstrate how easy it is to use TPU 😀, I also like the approach PL has taken in terms of organising the code. Anyone who writes PyTorch code ends up writing the same boiler plate over-and-over again, apart from the Model. 

---

@date: 03-Sep-2020 | @author: katnoria

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import transforms, datasets, models, utils
from torch.utils.data import DataLoader

import pytorch_lightning as pl

In [3]:
def version_info(cls):
    print(f"{cls.__name__}: {cls.__version__}")

In [4]:
version_info(pl)

pytorch_lightning: 0.9.0


```
{'conv_blocks': 5,
 'dropout': 0.0,
 'filters_0': 128,
 'filters_1': 160,
 'filters_2': 160,
 'filters_3': 192,
 'filters_4': 224,
 'hidden_size': 80,
 'learning_rate': 0.0025359172395390105,
 'pooling_0': 'max',
 'pooling_1': 'max',
 'pooling_2': 'max',
 'pooling_3': 'max',
 'pooling_4': 'max',
 'tuner/bracket': 2,
 'tuner/epochs': 30,
 'tuner/initial_epoch': 10,
 'tuner/round': 2,
 'tuner/trial_id': 'b780a5e9191d55c360ad5be6040decc4'}
```

In [19]:
class CIFARTenLitModel(pl.LightningModule):
    """CIFAR10 Model"""
    def __init__(self, backbone):
        super().__init__()
        self.backbone = backbone
        self.fc1 = nn.Linear(self.backbone.fc.out_features, 10)
#         nn.AvgPool2d()

    def forward(self, x):
        x = self.backbone(x)
        out = self.fc1(x)
        return out
    
    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = F.cross_entropy(y_hat, y)
        return pl.TrainResult(loss)
    
    def configure_optimizers(self):
        return optim.Adam(self.parameters(), lr=0.02)

# Dataset

In [6]:
tfms = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(tuple([0.5]*3), tuple([0.5]*3))
])

In [7]:
train_ds = datasets.CIFAR10(
    root="./data", 
    train=True,
    download=True,
    transform=tfms
)
train_loader = DataLoader(train_ds, batch_size=128, shuffle=True, num_workers=8)

Files already downloaded and verified


In [8]:
test_ds = datasets.CIFAR10(
    root="./data", 
    train=False,
    download=True,
    transform=tfms
)
test_loader = DataLoader(test_ds, batch_size=16, shuffle=False, num_workers=2)

Files already downloaded and verified


# Train

In [12]:
backbone = models.resnet50(pretrained=True)

In [ ]:
trainer = pl.Trainer()

In [20]:
model = CIFARTenLitModel(backbone)

GPU available: True, used: False
TPU available: False, using: 0 TPU cores


In [22]:
trainer.fit(model, train_loader)

/home/ashish/miniconda3/envs/torchv1/lib/python3.6/site-packages/pytorch_lightning/utilities/distributed.py:37: UserWarning: Could not log computational graph since the `model.example_input_array` attribute is not set or `input_array` was not given
  warnings.warn(*args, **kwargs)

  | Name     | Type   | Params
------------------------------------
0 | backbone | ResNet | 25 M  
1 | fc1      | Linear | 10 K  


RuntimeError: Given input size: (2048x1x1). Calculated output size: (2048x-5x-5). Output size is too small